In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


In [2]:
tf.test.is_gpu_available()

True

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [5]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.backend as K

In [6]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD

In [7]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

In [8]:
ImgTrainGen = image.ImageDataGenerator(preprocessing_function=vgg19.preprocess_input, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2,
                                  rotation_range=0.2,
                                  fill_mode='nearest',
                                  horizontal_flip=True, 
#                                   validation_split=1/6.
                                      )

Train = ImgTrainGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')

ImgTestGen = image.ImageDataGenerator(preprocessing_function=vgg19.preprocess_input, validation_split=1/6.)

Test  = ImgTestGen.flow_from_directory(join(prefix, imgfix, 'Test' ), 
                                       target_size=(224, 224), 
                                       class_mode='sparse', 
                                       batch_size=32, 
                                       shuffle=False, 
                                       interpolation='nearest')

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [9]:
def MultiTaskDataGen(ImgGen):
    while True:
        X = ImgGen.next()
        yield [X[0], X[1]], [X[1], X[1], X[1]]
        
TrainData = MultiTaskDataGen(Train)
# ValidData = MultiTaskDataGen(Valid)
TestData  = MultiTaskDataGen(Test)

# VGG Evaluation

In [10]:
model_vgg = models.load_model('vgg19_dfc120_40_6751')

In [11]:
res_vgg = model_vgg.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)

In [19]:
print('Losses of the fine tuned VGG model',res_vgg[0])
print('Accuracy of the fine tuned VGG model',res_vgg[1])

('Losses of the fine tuned VGG model', 1.949572550574869)
('Accuracy of the fine tuned VGG model', 0.6751748251748252)


# InceptionV3 Evaluation

In [13]:
model_inceptionV3 = models.load_model('vgg19_dfc120_40_6751')

In [14]:
res_inceptionV3 = model_inceptionV3.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)

In [15]:
print('Losses of the fine tuned InceptionV3 model',res_inceptionV3[0])
print('Accuracy of the fine tuned InceptionV3 model',res_inceptionV3[1])

('Losses of the fine tuned InceptionV3 model', 1.949572550574869)
('Accuracy of the fine tuned InceptionV3 model', 0.6751748251748252)


# DenseNet121 Evaluation

In [16]:
model_densenet121 = models.load_model('densenet121_dfc120_120_8059')

In [17]:
res_densenet121 = model_densenet121.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)

In [18]:
print('Losses of the fine tuned Densenet121 model',res_densenet121[0])
print('Accuracy of the fine tuned Densenet121 model',res_densenet121[1])

('Losses of the fine tuned Densenet121 model', 13.217681261431643)
('Accuracy of the fine tuned Densenet121 model', 0.009557109557109558)
